In [1]:
# %pip install z3-solver
from z3 import *
import re
import black
import jupyter_black
from icecream import ic

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)

In [3]:
# Problem from
# https://artofproblemsolving.com/wiki/index.php?title=2017_AMC_12A_Problems/Problem_1
# solution from Chapter 10 in https://sat-smt.codes/SAT_SMT_by_example.pdf


single, box_three, box_five = Ints("single box_three box_five")
pop_total = Int("pop_total")
cost_total = Int("cost_total")

s = Optimize()

s.add(pop_total == single * 1 + box_three * 3 + box_five * 5)
s.add(cost_total == single * 1 + box_three * 2 + box_five * 3)

s.add(cost_total == 8)

# Gives wrong answer unless these are added
s.add(single >= 0)
s.add(box_three >= 0)
s.add(box_five >= 0)

s.maximize(pop_total)

s.check(), s.model()

(sat,
 [box_five = 2,
  box_three = 1,
  cost_total = 8,
  pop_total = 13,
  single = 0])

In [45]:
# AoC 2015, day 15.
# Inspired by
# https://www.reddit.com/r/adventofcode/comments/q58dpl/2015_day_15_part_12_python_my_z3_solution/
def z3_property(index):
    val = sum(ingredients[i][index] * Spoons[i] for i, _ in enumerate(ingredients))
    return If(val > 0, val, 0)


def z3_score():
    return z3_property(0) * z3_property(1) * z3_property(2) * z3_property(3)


# My input - doesn't complete after several hours running time
# Frosting: capacity 4, durability -2, flavor 0, texture 0, calories 5
# Candy: capacity 0, durability 5, flavor -1, texture 0, calories 8
# Butterscotch: capacity -1, durability 0, flavor 5, texture 0, calories 6
# Sugar: capacity 0, durability 0, flavor -2, texture 2, calories 1
# frosting = (4, -2, 0, 0, 5)
# candy = (0, 5, -1, 0, 8)
# butterscotch = (-1, 0, 5, 0, 6)
# sugar = (0, 0, -2, 2, 1)
# ingredients = (frosting, candy, butterscotch, sugar)

# Input from reddit, takes 2-6 min to complete
sprinkles = (5, -1, 0, 0, 5)
peanutbutter = (-1, 3, 0, 0, 1)
frosting = (0, -1, 4, 0, 6)
sugar = (-1, 0, 0, 2, 8)
ingredients = (sprinkles, peanutbutter, frosting, sugar)

properties = ["capacity", "durability", "flavor", "texture"]
Spoons = [Int(f"{p}") for p in properties]
score = Int("score")

s = Optimize()
s.add(Sum(Spoons) == 100)
s.add(*(Spoons[p] > 0 for p, _ in enumerate(properties)))
s.add(score == z3_score())
s.maximize(score)
s.check(), s.model()

(sat,
 [flavor = 18,
  durability = 35,
  texture = 19,
  score = 13882464,
  capacity = 28])

In [47]:
# AOC 2018 day 23, part 2
def manhattan_distance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1]) + abs(a[2] - b[2])


def zabs(x):
    return If(x >= 0, x, -x)


lines = open("2018/23.txt").read().splitlines()
bots = [
    tuple(int(x) for x in re.findall("-?\d+", line)) for line in lines
]  # x, y, z, r

(x, y, z) = Ints("x y z")
in_ranges = [Int(f"in_range_{i}") for i, _ in enumerate(bots)]
range_count = Int("range_count")
s = Optimize()
for i, bot in enumerate(bots):
    nx, ny, nz, nrng = bot
    s.add(in_ranges[i] == If(zabs(x - nx) + zabs(y - ny) + zabs(z - nz) <= nrng, 1, 0))

s.add(range_count == sum(in_ranges))
dist_from_zero = Int("dist")
s.add(dist_from_zero == zabs(x) + zabs(y) + zabs(z))
s.maximize(range_count)
s.minimize(dist_from_zero)
s.check()
print("Part 2:", s.model().eval(dist_from_zero))

Part 2: 97816347


In [ ]:
# AOC 2018 day 23, part 2
def manhattan_distance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1]) + abs(a[2] - b[2])


def zabs(x):
    return If(x >= 0, x, -x)


lines = open("2018/23.txt").read().splitlines()
bots = [
    tuple(int(x) for x in re.findall("-?\d+", line)) for line in lines
]  # x, y, z, r

(x, y, z) = Ints("x y z")
in_ranges = [Int(f"in_range_{i}") for i, _ in enumerate(bots)]
range_count = Int("range_count")
s = Optimize()
for i, bot in enumerate(bots):
    nx, ny, nz, nrng = bot
    s.add(in_ranges[i] == If(zabs(x - nx) + zabs(y - ny) + zabs(z - nz) <= nrng, 1, 0))

s.add(range_count == sum(in_ranges))
dist_from_zero = Int("dist")
s.add(dist_from_zero == zabs(x) + zabs(y) + zabs(z))
s.maximize(range_count)
s.minimize(dist_from_zero)
s.check()
print("Part 2:", s.model().eval(dist_from_zero))

In [4]:
# AOC 2022 Day 21: Monkey Math using z3
import z3
from parse import parse
def solve(solve_for, lines):
    s = z3.Solver()
    values = {}
    for line in lines:
        if p := parse("{}: {:d}", line):
            monkey, value = p
            values[monkey] = z3.Int(monkey)
            if monkey != "humn" or solve_for != "humn":
                s.add(values[monkey] == value)
        elif p := parse("{}: {} {} {}", line):
            monkey, left, operand, right = p
            values[monkey] = z3.Int(monkey)
            values[left] = z3.Int(left)
            values[right] = z3.Int(right)
            if solve_for == "humn" and monkey == "root":
                s.add(values[left] == values[right])
            elif operand == "+":
                s.add(values[monkey] == values[left] + values[right])
            elif operand == "-":
                s.add(values[monkey] == values[left] - values[right])
            elif operand == "*":
                s.add(values[monkey] == values[left] * values[right])
            elif operand == "/":
                s.add(values[monkey] == values[left] / values[right])
    s.check()
    return s.model().eval(values[solve_for])


lines = open('2022/21.txt').read().splitlines()
print(f'Part 1: {solve("root", lines)}')  # 80326079210554
print(f'Part 2: {solve("humn", lines)}')  # 3617613952378

Part 1: 80326079210554
Part 2: 3617613952378
